In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/

/content


# 0. 파일 로드

In [ ]:
import os
import json
from tqdm import tqdm  # tqdm 임포트

BASE_DIR = "/content" # 드라이브 주소 수정

In [ ]:
CHUNK_SIZE = 1000
MAX_NEW_TOKENS = 600
CHUNK_OVERLAP = 100 # New: Overlap for text chunking

META_BASE_DIR = os.path.join(BASE_DIR, "meta") # 폴더에 맞게 수정
os.makedirs(META_BASE_DIR, exist_ok=True)

In [ ]:
META_BASE_DIR

'/content/meta'

# model load

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B-Instruct-2507"
# model_name = "Qwen/Qwen3-30B-A3B-Instruct-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

# Prompt

In [ ]:
def generate_prompt(text):
    return f"""
아래 텍스트는 의료 지식 전반(시술, 질환, 영양, 약물, 생리, 예방 등)에 대한 설명입니다.
텍스트의 성격에 맞게 metadata를 JSON으로 생성하세요.

[공통 규칙]
- 시술 관련 내용이 없을 경우, 시술 항목은 빈 배열([])로 두되
  의료 일반 지식에 해당하는 항목은 적극적으로 추출하세요.
- 텍스트에 명확히 등장하지 않는 정보는 추측하지 마세요.
- output으로 metadata만 출력해.

[Metadata 생성 기준]

1. keywords
- 텍스트의 핵심 의료 키워드
- 시술, 질환, 영양소, 약물, 생리 물질, 항산화제, 보충제 등 모두 포함 가능
- 영문, snake_case

2. procedure_type
- 실제 의료 시술, 치료, 처치가 명시된 경우만 추출
- 예: epidural_block, nerve_block, injection, surgery
- 시술이 없으면 []

3. target_area
- 치료 또는 주요 논의 대상이 되는 인체 부위
- 한글 → 영어 표준 용어로 변환
  - 경추 → cervical
  - 흉추 → thoracic
  - 요추 → lumbar
  - 미추/천추 → sacral
- 인체 부위가 아닌 경우 []

4. symptoms
- 환자 증상, 질환의 주요 임상 증상
- 또는 해당 의료 지식이 다루는 건강 문제
- 영문, snake_case
- 증상 개념이 없으면 []

[출력 형식 예시]
{{
  "keywords": ["selenium", "vitamin_e", "antioxidant"],
  "procedure_type": [],
  "target_area": [],
  "symptoms": []
}}

텍스트:
{text}
"""


## 병렬처리

In [ ]:
# !rm -rf /content/meta/

In [ ]:
pwd

'/content'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

def list_all_chunk_files(chunk_store_dir: str):
    paths = []
    for root, _, files in os.walk(chunk_store_dir):
        for fn in files:
            # done은 제외하고 원본 chunk만 대상
            if "_chunk_" in fn and fn.endswith(".json") and not fn.endswith("_done.json"):
                paths.append(os.path.join(root, fn))
    paths.sort()
    return paths

class ChunkFileDataset(Dataset):
    def __init__(self, chunk_files):
        self.chunk_files = chunk_files

    def __len__(self):
        return len(self.chunk_files)

    def __getitem__(self, idx):
        path = self.chunk_files[idx]

        # 이미 done 파일이 있으면 None 리턴해서 스킵
        done_path = path.replace(".json", "_done.json")
        if os.path.exists(done_path):
            return None

        with open(path, "r", encoding="utf-8") as f:
            item = json.load(f)

        item["_chunk_path"] = path
        item["_done_path"] = done_path
        return item

def collate_keep_valid(batch):
    return [x for x in batch if x is not None]

def safe_json_loads(text: str):
    s = text.strip()
    if s.startswith("```"):
        s = s.strip("`").replace("json", "").strip()
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        s2 = s.replace('"null"', 'null').replace("\\", "")
        try:
            return json.loads(s2)
        except json.JSONDecodeError:
            return None

## 강사님

In [ ]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_영문_국제기관 가이드라인/" "/content/meta"

In [ ]:
CHUNK_STORE_DIR = "/content/meta/TS_영문_국제기관 가이드라인/drive"

In [ ]:
import os
import json
import re
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm


@torch.inference_mode()
def infer_chunks_to_done_files(chunk_store_dir: str, batch_size: int = 16, MAX_NEW_TOKENS: int = 16):
    chunk_files = list_all_chunk_files(chunk_store_dir)

    ds = ChunkFileDataset(chunk_files)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,          # Colab 안정성
        collate_fn=collate_keep_valid
    )

    model.eval()

    processed = 0
    skipped_batches = 0
    parse_fail = 0
    batch_errors = 0

    pattern = re.compile(r"\{[\s\S]*?\}")  # JSON 후보 추출용

    for batch in tqdm(dl, desc="Pass B: 청크 배치 추론 -> done 저장"):
        if len(batch) == 0:
            skipped_batches += 1
            continue

        try:
            # 1) 프롬프트 구성
            texts = []
            for item in batch:
                prompt_text = generate_prompt(item["content"])
                messages = [{"role": "user", "content": prompt_text}]
                text = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True,
                )
                texts.append(text)

            # 2) 배치 토크나이즈
            model_inputs = tokenizer(
                texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
            ).to(model.device)

            # 3) 배치 generate
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False
            )

            # generate 정상 여부 1차 검증
            if generated_ids is None or generated_ids.ndim != 2:
                raise RuntimeError("model.generate 결과가 비정상입니다")

            # 4) 디코딩
            decoded_texts = tokenizer.batch_decode(
                generated_ids,
                skip_special_tokens=True
            )

            if len(decoded_texts) != len(batch):
                raise RuntimeError("batch size와 decoded_texts 길이가 불일치합니다")

            # 5) 샘플별 처리
            for item, decoded_text in zip(batch, decoded_texts):
                matches = list(pattern.finditer(decoded_text))

                if len(matches) == 1:
                    parse_fail += 1
                    # print("[JSON 미검출]")
                    # print(decoded_text)
                    # print("-" * 80)
                    continue

                json_candidate = matches[-1].group(0)

                try:
                    obj = json.loads(json_candidate)
                except Exception:
                    parse_fail += 1
                    # print("[JSON 파싱 실패]")
                    # print(decoded_text)
                    # print("-" * 80)
                    continue

                # 6) done 객체 구성
                done_obj = {
                    **item,
                    "chunk_id": item["chunk_id"],
                    "content": item["content"],
                    "keywords": obj.get("keywords", []),
                    "procedure_type": obj.get("procedure_type", []),
                    "target_area": obj.get("target_area", []),
                    "symptoms": obj.get("symptoms", [])
                }

                with open(item["_done_path"], "w", encoding="utf-8") as f:
                    json.dump(done_obj, f, ensure_ascii=False, indent=2)

                processed += 1

        except Exception as e:
            batch_errors += 1
            print("[배치 오류]", str(e))
            continue

    print("\n[Pass B 결과 요약]")
    print("processed done chunks:", processed)
    print("parse_fail chunks:", parse_fail)
    print("batch_errors:", batch_errors)
    if skipped_batches:
        print("skipped empty batches:", skipped_batches)


# 실행
infer_chunks_to_done_files(CHUNK_STORE_DIR, batch_size=100, MAX_NEW_TOKENS = 600)


Pass B: 청크 배치 추론 -> done 저장:   0%|          | 0/40 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Pass B: 청크 배치 추론 -> done 저장: 100%|██████████| 40/40 [1:28:36<00:00, 132.90s/it]


[Pass B 결과 요약]
processed done chunks: 2864
parse_fail chunks: 1087
batch_errors: 0


In [ ]:
START_TARGET_DIR = "/content/meta/TS_영문_국제기관 가이드라인/drive"

done_count = 0
not_done_cnt = 0

for root, _, files in os.walk(START_TARGET_DIR):
    for fn in files:
        if fn.endswith(".json") and "done" in fn:
            done_count += 1
        not_done_cnt += 1

print(f"done 이 포함된 JSON 파일 개수: {done_count}")
print(f"done 이 포함되지 않은 JSON 파일 개수: {not_done_cnt}")

done 이 포함된 JSON 파일 개수: 653
done 이 포함되지 않은 JSON 파일 개수: 1405


In [ ]:
# 주소 둘다 바꿔줘야함
os.makedirs("/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서", exist_ok=True)
!cp -r -v "/content/meta" "/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서"

'/content/meta' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/meta'
'/content/meta/cid_1605_1_9772_chunk_000.json' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/meta/cid_1605_1_9772_chunk_000.json'
'/content/meta/cid_1605_1_9772_chunk_001.json' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/meta/cid_1605_1_9772_chunk_001.json'
'/content/meta/cid_1605_1_9772_chunk_002.json' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/meta/cid_1605_1_9772_chunk_002.json'
'/content/meta/cid_1605_1_9772_chunk_003.json' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/meta/cid_1605_1_9772_chunk_003.json'
'/content/meta/cid_1605_1_9772_chunk_004.json' -> '/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta

In [ ]:
# from google.colab import runtime
# runtime.unassign()

In [ ]:
END_TARGET_DIR = "/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서"

done_count = 0
not_done_cnt = 0
done_files = []

for root, _, filenames in os.walk(END_TARGET_DIR):
    for fn in filenames:
        if not fn.endswith(".json"):
            continue

        if "done" in fn:
            done_count += 1
            if len(done_files) < 10:
                done_files.append(fn)
        else:
            not_done_cnt += 1

print(f"done 이 포함된 JSON 파일 개수: {done_count}")
print(f"done 이 포함되지 않은 JSON 파일 개수: {not_done_cnt}")


done 이 포함된 JSON 파일 개수: 3849
done 이 포함되지 않은 JSON 파일 개수: 5085


In [ ]:
done_files

['cid_3815_1_chunk_000_done.json',
 'cid_3815_1_chunk_001_done.json',
 'cid_3815_1_chunk_002_done.json',
 'cid_3816_1_7762_chunk_000_done.json',
 'cid_3816_1_7762_chunk_001_done.json',
 'cid_3816_1_7762_chunk_002_done.json',
 'cid_3816_1_chunk_000_done.json',
 'cid_3816_1_chunk_001_done.json',
 'cid_3817_1_4492_chunk_000_done.json',
 'cid_3817_1_4492_chunk_001_done.json']

In [ ]:
for path in done_file_paths[:10]:  # 너무 많으면 출력이 과도해짐
    print("=" * 80)
    print(f"파일 경로: {path}")

    with open(path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    print(raw_text)


파일 경로: /content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/meta/TS_국문의학 교과서/drive/cid_34782_1_4382_chunk_000_done.json
{
  "src_path": "/content/drive/MyDrive/Colab Notebooks/LikeLion/실전 프로젝트 2/DATA/TS_국문_학술 논문 및 저널/cid_34782_1_4382.json",
  "ts_folder": "drive",
  "file_name": "cid_34782_1_4382.json",
  "base": "cid_34782_1_4382",
  "chunk_id": 0,
  "content": "2004년도 보건복지부 지원으로 수행된 \"응급의료 기본계획 수립 및 응급의료 운영체계 평가\" 연구 자료를 기반으로 한 연구입니다. 응급환자의 진료 결과를 향상시키기 위해서는 현장에서 환자의 상태를 정확히 평가하고, 필요한 응급처치를 적시에 적절히 시행하는 것이 중요합니다. 응급구조사 제도 시행 이후 병원 전 단계에서의 응급처치 시행률, 적절성, 의사지도 시행률 등을 평가한 연구들이 보고되었으나, 기존 국내 연구들은 다음과 같은 한계점을 가지고 있었습니다. 첫째, 구급활동일지와 응급실 의무기록 자료의 질적 수준이 낮아 신뢰성 높은 자료를 활용할 필요성이 있었습니다. 둘째, 응급환자의 병원 전 외식 상태 및 생체징후 측정에 대한 평가가 제한적이었습니다. 기존 연구에서는 구급활동일지와 응급실 기록에 기재된 외식 상태의 일치율을 산출하여 평가했으나, 현장에서 병원 전 외식 상태를 측정한 후 병원 도착까지의 시간을 고려하지 못한 한계가 있었습니다. 셋째, 구급인력의 유형에 따른 응급처치 적절성이 평가된 바 없었습니다. 구급인력은 1급 및 2급 응급구조사, 간호사, 기타 인력으로 구분되며, 이들 간의 수행능력 차이가 있다면 이를 바탕으로 정책적 지원이 가능할 것입니다. 넷째, 어떤 지도 의사로부터 의료지도를

# tmp

In [ ]:
# import re

# @torch.inference_mode()
# def infer_chunks_to_done_files(chunk_store_dir: str, batch_size: int = 16):
#     chunk_files = list_all_chunk_files(chunk_store_dir)

#     ds = ChunkFileDataset(chunk_files)
#     dl = DataLoader(
#         ds,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=0,          # 코랩 안정성
#         collate_fn=collate_keep_valid
#     )

#     model.eval()

#     processed = 0
#     skipped_batches = 0
#     parse_fail = 0
#     batch_errors = 0

#     for batch in tqdm(dl, desc="Pass B: 청크 배치 추론 -> done 저장"):
#         if len(batch) == 0:
#             skipped_batches += 1
#             continue

#         # 1) 프롬프트 구성
#         texts = []
#         for item in batch:
#             prompt_text = generate_prompt(item["content"])
#             messages = [{"role": "user", "content": prompt_text}]
#             text = tokenizer.apply_chat_template(
#                 messages,
#                 tokenize=False,
#                 add_generation_prompt=True,
#             )
#             texts.append(text)
#             # content만 추출

#         # 2) 배치 토크나이즈
#         model_inputs = tokenizer(
#             texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#         ).to(model.device)


#         # 3) 배치 generate
#         generated_ids = model.generate(
#             **model_inputs,
#             max_new_tokens=MAX_NEW_TOKENS,
#             do_sample=False
#         )

#         # 4) 디코딩
#         decoded_texts = tokenizer.batch_decode(
#             generated_ids,
#             skip_special_tokens=True
#         )


#         # # 4) 샘플별 디코딩 (각 입력 길이만큼 잘라냄)
#         # input_lens = model_inputs["attention_mask"].sum(dim=1).tolist()

#         # for item, gen_ids, in_len in zip(batch, generated_ids, input_lens):
#         #     out_ids = gen_ids[int(in_len):].tolist()
#         #     generated_output = tokenizer.decode(out_ids, skip_special_tokens=True)

#         #     metadata = safe_json_loads(generated_output)
#         #     if metadata is None:
#         #         cleaned = re.sub(r'(?s).*?\bassistant\s*', '', generated_output)
#         #         metadata = safe_json_loads(cleaned)

#         #     if metadata is None:
#         #         parse_fail += 1
#         #         print(cleaned)
#         #         break

#         pattern = re.compile(r"\{[\s\S]*?\}") # 모든 딕셔너리 형태 잡기
#         for item, decoded_text in zip(batch, decoded_texts):
#             matches = list(pattern.finditer(decoded_text))

#             if len(matches) != 2:
#                 print(decoded_text)

#             json_candidate = matches[-1].group(0) if matches else None # 마지막것만
#             if json_candidate is None:
#                 print(decoded_text)
#                 print("----------------------------------------")

#             try:
#                 obj = json.loads(json_candidate)
#                 # 5) "청크 + 추론결과" 합쳐서 done 저장
#                 done_obj = {
#                     **item,                 # 원본 메타
#                     "chunk_id": item["chunk_id"],
#                     "content": item["content"],
#                     # 추론 결과를 합치는 부분 (원하던 구조)
#                     "keywords": obj.get("keywords", []),
#                     "procedure_type": obj.get("procedure_type", []),
#                     "target_area": obj.get("target_area", []),
#                     "symptoms": obj.get("symptoms", [])
#                 }

#                 with open(item["_done_path"], "w", encoding="utf-8") as f:
#                     json.dump(done_obj, f, ensure_ascii=False, indent=2)
#             except:
#                 print(decoded_text)

#             processed += 1




#     print("\n[Pass B]")
#     print("processed done chunks:", processed)
#     print("parse_fail chunks:", parse_fail)
#     print("batch_errors chunks:", batch_errors)
#     if skipped_batches:
#         print("skipped empty batches:", skipped_batches)

# # 실행
# infer_chunks_to_done_files(CHUNK_STORE_DIR, batch_size=30)

## try

In [ ]:
# import os
# import json
# import torch
# from torch.utils.data import DataLoader
# from tqdm import tqdm

# @torch.inference_mode()
# def infer_chunks_to_done_files(
#     chunk_store_dir: str,
#     batch_size: int = 8,
#     max_new_tokens: int = 256
# ):
#     chunk_files = list_all_chunk_files(chunk_store_dir)

#     ds = ChunkFileDataset(chunk_files)
#     dl = DataLoader(
#         ds,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=0,          # Colab 안정성
#         collate_fn=collate_keep_valid
#     )

#     model.eval()

#     processed = 0

#     for batch in tqdm(dl, desc="Batch inference"):
#         if len(batch) == 0:
#             continue

#         texts = [generate_prompt(item["content"]) for item in batch]

#         inputs = tokenizer(
#             texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True
#         ).to(model.device)

#         generated_ids = model.generate(
#             **inputs,
#             max_new_tokens=max_new_tokens,
#             do_sample=False
#         )

#         decoded_texts = tokenizer.batch_decode(
#             generated_ids,
#             skip_special_tokens=True
#         )

#         for item, decoded in zip(batch, decoded_texts):
#             meta = safe_json_loads(decoded)

#             if meta is None:
#                 # JSON 파싱 실패 시 최소한의 구조라도 저장
#                 meta = {
#                     "keywords": [],
#                     "procedure_type": [],
#                     "target_area": [],
#                     "symptoms": []
#                 }

#             done_obj = {
#                 # --- chunk 기본 메타 ---
#                 "src_path": item.get("src_path", ""),
#                 "ts_folder": item.get("ts_folder", ""),
#                 "file_name": item.get("file_name", ""),
#                 "base": item.get("base", ""),
#                 "chunk_id": item.get("chunk_id"),
#                 "content": item.get("content", ""),

#                 # --- 의료 메타데이터 ---
#                 "keywords": meta.get("keywords", []),
#                 "procedure_type": meta.get("procedure_type", []),
#                 "target_area": meta.get("target_area", []),
#                 "symptoms": meta.get("symptoms", [])
#             }

#             with open(item["_done_path"], "w", encoding="utf-8") as f:
#                 json.dump(done_obj, f, ensure_ascii=False, indent=2)

#             processed += 1

#     print(f"총 처리 완료 chunk 수: {processed}")


In [ ]:
# infer_chunks_to_done_files(CHUNK_STORE_DIR, batch_size=140)

In [ ]:
# torch.cuda.empty_cache()